In [2]:
import  numpy as np
from scipy.linalg import expm,logm, sinm, cosm

from qiskit import BasicAer, Aer,execute, IBMQ
from qiskit import QuantumRegister ,ClassicalRegister,QuantumCircuit
from qiskit.providers.aer import extensions 

In [3]:
from qmatches.initialize import init_xo

from qmatches.mat_tools.matrix import *
from qmatches.mat_tools.space_builder import COp

In [129]:
# Setup

#initial number of qubits
nr_q=2 

#2n hermitian operators on n-qubits JW  
c_op=COp(nr_q)

c_op.info()

size: 2
nr. elements: 4


ethiket: c_0
labels: ['XI']
coef: [(1+0j)]
matrix: 
 [[0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]]
#################################################
ethiket: c_1
labels: ['YI']
coef: [(1+0j)]
matrix: 
 [[0.+0.j 0.+0.j 0.-1.j 0.-0.j]
 [0.+0.j 0.+0.j 0.-0.j 0.-1.j]
 [0.+1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+1.j 0.+0.j 0.+0.j]]
#################################################
ethiket: c_2
labels: ['ZX']
coef: [(1+0j)]
matrix: 
 [[ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -0.+0.j -1.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j -0.+0.j]]
#################################################
ethiket: c_3
labels: ['ZY']
coef: [(1+0j)]
matrix: 
 [[ 0.+0.j  0.-1.j  0.+0.j  0.-0.j]
 [ 0.+1.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.-0.j -0.+0.j  0.+1.j]
 [ 0.+0.j  0.+0.j -0.-1.j -0.+0.j]]
#################################################


## Initiale state 

In [130]:
def compress_denisty(dens,op):
    '''
    1/nr_q[1-iC(dens)]
    '''
    nrq=int(np.log2(len(dens)))
    ide=np.identity(2*nrq,dtype=np.complex128)
    Co=corelation(dens,op)
    for i in range(len(Co)):
        for j in range(len(Co)):
            Co[i][j]=1j*(Co[i][j])
    return (1/(2*nrq))*(ide+Co)

In [131]:
nr_e=0

# intitial state can be : |0000> , |0001>,|0011>,|0111>,|1111> nr of ones =nr_e
initial_state=init_xo(nr_q,nr_e)

print('Initial state:')

print('Iniitial density:\n',initial_state.density)

Initial state:
Iniitial density:
 [[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [132]:
compres_density_mixt=compress_denisty(initial_state.density,c_op)
S_corelation=corelation(init_xo(nr_q,2).density, c_op)



print('Corelation Matrix:')
print_paili_decompose(S_corelation)
print(S_corelation)

print()
print('Compres matrix:')
print_paili_decompose(compres_density_mixt)
print(compres_density_mixt)

Corelation Matrix:
IY:1j
[[0j, (1+0j), 0j, 0j], [(-1+0j), 0j, 0j, 0j], [0j, 0j, 0j, (1+0j)], [0j, 0j, (-1+0j), 0j]]

Compres matrix:
II:(0.25+0j)
IY:(0.25+0j)
[[0.25+0.j   0.  -0.25j 0.  +0.j   0.  +0.j  ]
 [0.  +0.25j 0.25+0.j   0.  +0.j   0.  +0.j  ]
 [0.  +0.j   0.  +0.j   0.25+0.j   0.  -0.25j]
 [0.  +0.j   0.  +0.j   0.  +0.25j 0.25+0.j  ]]


### So far, I think that I did the right thing.

## Observable:

In [133]:
class obs_c():
    def __init__(self,a_mat,c_op):
        self.a_mat=a_mat
        self.c_op=c_op
    
    def matrix(self):
        s=np.zeros((2**c_op.size,2**c_op.size))
        for j in range(len(self.a_mat)):
            for k in range(len(self.a_mat)):
                if (self.a_mat[j][k]!=0):
                    s=s+self.a_mat[j][k]*np.matmul(c_op.elements[j],c_op.elements[k])
        return 1j/2*s

In [134]:
a_mat= np.zeros((2*nr_q,2*nr_q))
a_mat[-1][-2]=-1.0
a_mat[-2][-1]=1.0


A_obs=obs_c(a_mat,c_op)

print("Initial observable:\n",A_obs.matrix())
print("Initial observable decompose:\n")
print_paili_decompose(A_obs.matrix())
h,h_label=pauli_decompose(A_obs.matrix())

Initial observable:
 [[-1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  1.+0.j]]
Initial observable decompose:

IZ:(-1+0j)


In [135]:
def dec_vec(nr,nrq):
    r=np.zeros(2*nrq) 
    r[nr]=1
    return r

#A=1j/2(\sum_0^(2n-1)a_{j,k}c_jc_k)

#1.85 ->Did I code the formula correctly?
def compress_obs(obs):
    s=np.zeros((len(obs.c_op.elements),len(obs.c_op.elements)))
    for j in range(len(obs.a_mat)):
        for k in range(len(obs.a_mat)):
            if(j!=k):
                s=s+obs.a_mat[j][k]*np.outer(dec_vec(j,obs.c_op.size),dec_vec(k,obs.c_op.size))
    return 1j/2*s

In [136]:
A_compress=compress_obs(A_obs)

print("Compress observable:\n",A_compress)
print("Compress observable decompose:\n")
print_paili_decompose(A_compress)

Compress observable:
 [[ 0.+0.j   0.+0.j   0.+0.j   0.+0.j ]
 [ 0.+0.j   0.+0.j   0.+0.j   0.+0.j ]
 [ 0.+0.j   0.+0.j   0.+0.j   0.+0.5j]
 [ 0.+0.j   0.+0.j  -0.-0.5j  0.+0.j ]]
Compress observable decompose:

IY:(-0.25+0j)
ZY:(0.25+0j)


### 1.85 ->Did I code the 1.85 formula correctly?

## Gatas Compresion

->gate=e^{iwH} where H=i/2(\sum_{i,j}a_{ij}c_ic_j) a_{i,j} i!=j are rel  and a_{i,j}=-1a_{j,i}

In [137]:
# Derivation of matrix RJ
from qmatches.gates import GZB

In [138]:
theta=np.pi/7
gate=GZB(theta)
gate.mat

array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.43388374,  0.90096887,  0.        ],
       [ 0.        ,  0.90096887,  0.43388374,  0.        ],
       [ 0.        ,  0.        ,  0.        , -1.        ]])

In [139]:
C=c_op.elements
a=theta
k_1=0
k_2=1


H=1j*((1-np.cos(-a))/4*(np.matmul( C[2*k_1+1],C[2*k_1])-np.matmul(  C[2*k_1],    C[2*k_1+1]))
      +np.sin(a)/4*(np.matmul(     C[2*k_2],  C[2*k_1+1])-np.matmul(C[2*k_1+1],  C[2*k_2]))
      +(np.cos(-a)+1)/4*(np.matmul(C[2*k_2+1],C[2*k_2])-np.matmul(  C[2*k_2],    C[2*k_2+1]))
      -np.sin(a)/4*(np.matmul(     C[2*k_2+1],C[2*k_1])-np.matmul(  C[2*k_1],    C[2*k_2+1])))

H

array([[ 1.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j],
       [ 0.        +0.j, -0.90096887+0.j,  0.43388374+0.j,
         0.        +0.j],
       [ 0.        +0.j,  0.43388374+0.j,  0.90096887+0.j,
         0.        +0.j],
       [ 0.        +0.j,  0.        +0.j,  0.        +0.j,
        -1.        +0.j]])

The problem here is that wrote thr GZB matrix directly as a sum of c_op elements.
Do you know how I could find out H's value the right way? Or maybe is a trick  to find out Rj directly?

In [147]:
w_i=np.pi/2
U_i=expm(-0.5*w_i*(H))
U_i

array([[ 0.45593813+0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j],
       [ 0.        +0.j,  2.10725458+0.j, -0.3769022 +0.j,
         0.        +0.j],
       [ 0.        +0.j, -0.3769022 +0.j,  0.5419636 +0.j,
         0.        +0.j],
       [ 0.        +0.j,  0.        +0.j,  0.        +0.j,
         2.19328005+0.j]])